In [ ]:

using Random: seed!

In [ ]:

myseed = 10
seed!(myseed)

In [ ]:

rand(5)

In [ ]:

myseed = 10
seed!(myseed)   # re-seeding
rand(5)         # outputs same sequence of random numbers

In [ ]:

using Statistics: mean, std

In [ ]:

using PyPlot

In [ ]:

""""
    pi_est = mcpi(m)

Monte Carlo calculation of pi
"""
function mcpi(m)
    s = 0.0
    for i = 1:m
        x, y = rand(2)
        if x^2 + y^2 < 1
            s += 1.0
        end
    end
    return 4 * s/m
end

In [ ]:

mcpi(1000000000)

In [ ]:

m = 2^20
k = 256;

In [ ]:

seed = 1
seed!(seed)
res = zeros(k)
for n = 1:k
    @info n
    res[n] = mcpi(m)
end

In [ ]:

res

In [ ]:

hist(res)

In [ ]:

n = 12                  # the number of experiments
nps = zeros(Int64, n)   # the numbers of MC steps in each experiment
pis = zeros(n)          # the estimations of `pi` obtained in each experiment 
pistds = zeros(n);      # the stds of estimations

In [ ]:

k = 256               # the number of MC runs with a fixed number of MC steps
res = zeros(k)        # storage for the results of `k' MC runs
ns = zeros(Int64, k); # working array of integers

In [ ]:

@time for i = 1:n
    nps[i] = 2^(i+10)
    @info i, nps[i]
    fill!(ns, nps[i])  # fill the array `ns` with values of np[i]
    res .= mcpi.(ns)   # run mce() k times and store the results to array `res`
    pis[i] = mean(res)
    pistds[i] = std(res, mean=pis[i])/sqrt(k)  # the stds
end

In [ ]:

pis

In [ ]:

loglog(nps, abs.(pis .- pi), label="True errors", marker="o", markersize=2, linestyle="none")
loglog(nps, pistds, label="Stds", marker="o", markersize=3, linestyle="none")
ylim(1e-6, 1e-1)
grid(true)
xlabel("MC steps")
ylabel("Absolute error")
title("Results of Monte Carlo calculations")
legend();

In [ ]:

function linear_regression(x, y)
    np = length(x)
    xbar = sum(x)/np
    ybar = sum(y)/np
    x2 = sum((x .- xbar) .^ 2)
    beta = sum((y .- ybar) .* (x .- xbar))/x2
    alpha = ybar - beta*xbar
    sigma = sqrt(sum((y .- alpha .- beta .* x) .^ 2)/((np - 2)*x2))
    return alpha, beta, sigma
end

In [ ]:

alpha, beta, sigma = linear_regression(log.(nps), log.(pistds));

In [ ]:

round(beta, sigdigits=3)

In [ ]:

round(sigma, sigdigits=3)

In [ ]:

loglog(nps, abs.(pis .- pi), label="True error", marker="o", markersize=2, linestyle="none")
loglog(nps, pistds, label="Std", marker="o", markersize=3, linestyle="none")
loglog(nps, exp.(alpha .+ beta .* log.(nps)), label="LSq fit std", linestyle="dashed")
grid(true)
xlabel("MC steps")
ylabel("Absolute error")
title("MC simulations results and linear regression fit")
legend();